In [1]:
import os
import pyodbc
import pandas as pd
import xlsxwriter
import math
from datetime import datetime
from os import walk
from IPython.core.debugger import set_trace


In [2]:
################################
# Function to run query in db
# Inputs: 
#       cnxn -> connection object to DB
#       query -> query string
#       params -> tuple of query parameters
# Output:
#       rows -> query result. Rows of data from DB
#################################

In [3]:
def run_query(cnxn,query,params):
    cursor = cnxn.cursor()
    cursor.execute(query,params)
    rows = []
    if "INSERT" not in query:
        rows = cursor.fetchall()
    return rows


In [4]:
################################
# Function to find index of cell (from excel data)
# where the value starts with "Date".
# This index will then be used to find the calendar, weather and consumption data
# Inputs: 
#       data -> excel data
#       total_rows -> num. rows in excel
#       i,j -> starting position of excel cells
# Output:
#       i -> Row index of "Date"
#################################

In [5]:
def find_index_Date(data,total_rows,i,j):
    found_Date = False
    while not found_Date:
        cell_value = str(data.iloc[i,j])
        if cell_value is not None:
            if cell_value.startswith("Date"):
                found_Date = True
            else:
                i+=1
                if i > total_rows-1:
                    i = -1 # To identify end of row is reached
                    found_Date = True
        else:
            i+=1
            if i > total_rows-1:
                i = -1 # To identify end of row is reached
                found_Date = True
    return i

In [6]:
################################
# Function to find index of cell (from excel data)
# where the value starts with "Actual".
# This index will then be used to find the energy profile data
# Inputs: 
#       data -> excel data
#       total_rows -> num. rows in excel
#       i,j -> starting position of excel cells
# Output:
#       i -> Row index of "Actual"
#################################

In [7]:
def find_index_Actual(data,total_cols):
    found_Actual = False
    i = 4
    j = 0
    while not found_Actual:
        cell_value = str(data.iloc[i,j])
        if cell_value is not None:
            if cell_value.startswith("Actual"):
                found_Actual = True
            else:
                j+=1
                if j > total_cols-1:
                    j = -1 # To identify end of row is reached
                    found_Actual = True
        else:
            i+=1
            if j > total_cols-1:
                j = -1 # To identify end of row is reached
                found_Actual = True
    return j

In [8]:
################################
# Function to find store id number (from excel data)
#
# Inputs: 
#       data -> excel data
#       total_rows -> num. rows in excel
#       i,j -> starting position of excel cells
# Output:
#       id_int -> integer value of branch
#################################     

In [9]:
def find_index_branch_id(data,total_rows,i,j):

    cell_value = str(data.iloc[i,j])
    profile_info = data.iloc[i,j] # Usually the string is "### - loc/XXXX)"
    profile_split = profile_info.split("loc/", 1) # Split the string with "loc/"
    id_str = profile_split[1].strip()
    id_str = id_str[:-1]
    return_param = id_str
    
    #found_branch_id = False
    #while not found_branch_id:
        
        #if "- loc/" in cell_value == True:
        #    print("True")
        #    profile_info = data.iloc[i,j] # Usually the string is "### - loc/XXXX)"
        #    cell_split = profile_info.split("loc/") # Split the string with "loc/"
        #    id_str = cell_split[0].strip() # second part of split string will be store id with ")", i.e. "XXXX)"
        #    id_int = int(id_str[:-1]) # remove last char ")"
        #    return_param = id_int
        #    found_branch_id = True
        #else:
        #    i+=1
        #    if i > total_rows-1:
        #        return_param = -1
        #        found_branch_id = True

                    
    return return_param

In [10]:
################################
# For the given value this function 
# tries to get date. If not it returns
# "None" value
################################

In [11]:
def get_date(val):
    date_val = None

    if type(val) is str:
        date_val = datetime.strptime(val,"%d/%m/%Y")
    else:
        date_val = val

    return date_val

In [12]:
################################
# For the given value this function 
# tries to get time. If not it returns
# "None" value
################################

In [13]:
def get_time(val):
    time_val = None
    
    if type(val) is str:
        time_val = datetime.strptime(val,"%H:%M:%S")        
    else:
        time_val = val

    return time_val

In [14]:
################################
# For the given value this function 
# tries to get percentage value if
# the input value is a string (e.g. '85%')
# If not it returns "None" value
################################

In [15]:
def get_percentage(val):
    percentage_val = None
    try:
        percentage_val = float(val)
    except:
        percentage_val = val[:-1] # From input value (e.g. '85%'), removes percentage
        percentage_val = percentage_val.replace(",","") # Found some percentage string values with ",". Just removing them
        percentage_val = float(percentage_val) / 100

    return percentage_val

In [16]:
################################
# For the given value this function 
# tries to get int value. If not it 
# returns 0
################################

In [17]:
def get_int(val):
    if val is None:
        return 0
    else:
        return int(val)

In [18]:
################################
# For the given value this function 
# tries to get float value. If not it 
# returns 0
################################

In [19]:
def get_float(val):
    if val is None:
        return 0
    else:
        return float(val)

In [20]:
## EDIT VERSION

# Parent directory of all excel files 
xlsx_files_root_dir = r"C:\Users\benfl\OneDrive - Flexible Power Systems Ltd\Documents\PCM\Load Profiles - 200120 (FROM SSL)" 

file_tag = "_complete.xlsx"
output_folder = "complete_profiles"
column_headings_path_file = r"C:\Users\benfl\OneDrive - Flexible Power Systems Ltd\Documents\PCM\Load Profile Headers 2.xlsx" 

# read headings 
column_headings = pd.read_excel(column_headings_path_file,keep_default_na=False)

In [21]:
xlsx_files_root_dir = r"C:\Users\benfl\OneDrive - Flexible Power Systems Ltd\Documents\PCM\Load Profiles - 200120 (FROM SSL)" 

# (21/01/20)  ensure that dates are properly aligned

column_headings_order = column_headings[column_headings.columns[0]]
data_offset = 6
old_branch_id = str(0)
count = 0
values_to_remove = ['Active', 'Setup']
dirpath = r"C:\Users\benfl\OneDrive - Flexible Power Systems Ltd\Load Profiles - PythonJoinTestingDelete\Super\10092018IslingtonSuper_0500"
# Fetches all files in directory and loops 
raw_input_row_location_of_header = 4

In [22]:
# - NEW - LATEST  (13/02/20)

column_headings_path_file = r"C:\Users\benfl\OneDrive - Flexible Power Systems Ltd\Documents\FPS Developer\SSL_data_cleaning_v3\Documentation\Profile_Schema.xlsx"
column_headings = pd.read_excel(string_new_headers, keep_default_na=False, index_col=0)
column_headings_order = column_headings_NEW.iloc[0]
new_index = list(range(0,column_headings_order.size))
column_headings_order.index = new_index

NameError: name 'string_new_headers' is not defined

In [25]:
xlsx_files_root_dir = r"C:\Users\benfl\OneDrive - Flexible Power Systems Ltd\Documents\PCM\Load Profiles - 200120 (FROM SSL)" 

# NEW - LATEST  (21/01/20)  ensure that dates are properly aligned

data_offset = 6
old_branch_id = str(0)
count = 0
values_to_remove = ['Active', 'Setup']
first_run = True
# Fetches all files in directory and loops thorough each file
for (dirpath, dirnames, filenames) in walk(xlsx_files_root_dir):
    for filename in filenames:
        
        # 0. Set any constants and skip if XXXX_complete 
        
        raw_input_row_location_of_header = 4
        
        # 1. skip if file is 'XXXX_complete.xlsx'
        
        if file_tag in filename:
            continue
        
        # 2. Load file and delete trailing columns

        print(filename)
        data = pd.read_excel(os.path.join(dirpath,filename),keep_default_na=False)
        total_rows = len(data)
        total_cols = len(data.columns)
        i=0
        j=0
        
        cols_to_drop = []
        for indx in range(1, data.shape[1]):
            if len(data.iloc[raw_input_row_location_of_header,indx]) == 0:
                cols_to_drop.append(indx)
        data.drop(data.columns[cols_to_drop], axis=1, inplace=True)
        
        
        # 3. Find store number
        
        branch_id = find_index_branch_id(data, total_cols, i, j)
        
        # 4. If the store is different to the previous loop (is now complete), remove any leftover strings, update headers and save
        
        try:
            if first_run == False and branch_id_prev != branch_id:
                # find unwanted string values cells and delete
                for value in values_to_remove:
                    listOfPos = list()
                    # Get bool dataframe with True at positions where the given value exists
                    result = complete_data_new.isin([value])
                    # Get list of columns that contains the value
                    seriesObj = result.any()
                    columnNames = list(seriesObj[seriesObj == True].index)
                    # Iterate over list of columns and fetch the rows indexes where value exists
                    for col in columnNames:
                        complete_data_new[col] = ''
                
                complete_data_new.columns = column_headings_order
                complete_data_new.to_excel(os.path.join(xlsx_files_root_dir + "/" + output_folder + "/" + str(branch_id_prev) + file_tag), index=False)
        except NameError:
            pass
        

        # 5. Find var_name
        
        # find key word 1 (first word) "Electricity"/"Heat"/"Gas" + " consumption/production"
        profile_info = str(data.iloc[0,0]) # find the profile information
        cell_split = profile_info.split(" at ", 1) # Split the string at " at "
        kw2 = cell_split[0].strip() # get profile type "key word 1"
        # check if profile is a store total, i.e. if branch id starts second string
        if  (cell_split[1].strip()).find(str(branch_id)) == 0:
            total_flag = True # if it is a total, set "key word 2 " as total 
            kw1 = "Store"
        else:
            total_flag = False # if it is not a total, set "key word 2 " as the variable 
            cell_split2 = cell_split[1].split(" (", 1)
            kw1 = cell_split2[0].strip()
        # create full variable name    
        var_name = kw1 + " " + kw2 + " (kW)"

        # 6. Find "Actual" column
        
        act_col = find_index_Actual(data, total_cols)

        # 7. Set to var_name
        
        data.iloc[raw_input_row_location_of_header,act_col] = var_name

        # 8. Make column names headers
        
        data = data.drop(data.index[[0,1,2,3]])
        new_header = data.iloc[0] #grab the first row for the header
        data = data[1:] #take the data less the header row
        data.columns = new_header #set the header row as the df header

        # 9. Check if XXXX_complete.xlsx exists
        
        complete_exists = False
        
        for (dirpath2, dirnames2, filenames2) in walk(os.path.join(xlsx_files_root_dir,output_folder)):
            if filenames2 == []:
                break
            for filename2 in filenames2:
                if str(filename2) == str(branch_id) + file_tag:
                    complete_exists = True
                    break

        # 9.1. If complete exists:
            # Open complete
            # Merge with current data
            
        if complete_exists == True:
            #data['Date'].astype('datetime64[ns]')
            data.to_excel(os.path.join(xlsx_files_root_dir + "/" + "tmp" + file_tag), index=False)
            data = pd.read_excel(os.path.join(xlsx_files_root_dir + "/" + "tmp" + file_tag),keep_default_na=False)
            complete_data = pd.read_excel(os.path.join(xlsx_files_root_dir + "/" + output_folder + "/" + str(branch_id) + file_tag))
            complete_data_new = pd.merge(complete_data,data,on='Date', how='outer') 
            # remove any trailing '_x' from variable names
            post_merge_headers = complete_data_new.columns
            cleaned_headers = list(map(lambda x: x.replace('_x',''),post_merge_headers))
            complete_data_new.columns = cleaned_headers
            
            # Find duplicated date rows, caused by clock changes producing duplicate time rows and merging doubling these
                # delete the ones which are incorrect mixes of the extra times due to clock change
            bool_duplicates_all = complete_data_new.duplicated(subset=['Date'], keep  = False)
            duplicate_rows = complete_data_new[bool_duplicates_all]

            to_remove = []
            for i in range(0, duplicate_rows.shape[0]):
                z = duplicate_rows.shape[0] - (i+1)
                if (z % 4) == 1 or (z % 4) == 2:
                    to_remove.append(z)
            duplicate_indx = [i for i, x in enumerate(bool_duplicates_all) if x]
            for i in to_remove:
                complete_data_new.drop(duplicate_indx[i], inplace=True)
            bool_duplicates_all = complete_data_new.duplicated(subset=['Date'], keep  = False)
            duplicate_rows = complete_data_new[bool_duplicates_all]
            duplicate_indx = [i for i, x in enumerate(bool_duplicates_all) if x]
            count = 2
            for z in range(0,len(duplicate_indx)):
                if ((z+1) % 2) == 0:
                    if count == 3:
                        i = duplicate_indx[z]
                        # swap rows i and i+1
                        b, c = complete_data_new.iloc[i], complete_data_new.iloc[i+1]
                        temp = complete_data_new.iloc[i].copy()
                        complete_data_new.iloc[i] = c
                        complete_data_new.iloc[i+1] = temp
                        count = 0
                count = count + 1       
            
        else:
             complete_data_new = data
            
        # 10. Make sure there are at least the required number of columns
        
        if complete_data_new.shape[1] < column_headings.shape[0]:
            numb_new_cols_req = column_headings.shape[0] - complete_data_new.shape[1]
            complete_data_new = complete_data_new.reindex(complete_data_new.columns.tolist() + ['new_var' + str(INDEX) for INDEX in range(column_headings.shape[0]-numb_new_cols_req,column_headings.shape[0])], axis=1)
            
            
            
        # 11. For each variable header:
            # Check to see if it is contained in the input headers
            # If contained:
                # Find location data should go
                # Copy column and header to that location
                # Delete column at old location
        
        for header in complete_data_new.columns: # for each header in the new merged set
            try:
                #print(header)
                header_loc = column_headings_order[column_headings_order == header].index[0] # -> try to find the location of that header in the set of expected headers
                #print(header + " has header_loc: " + str(header_loc))
            except ValueError:
                header_loc = -1 # if the header can't be found indicate this
                #print("Header is not found")
            except IndexError:
                header_loc = -1 # if the header can't be found indicate this
                #print("Header is not found")
            if header_loc != -1: # if the header is found
                names = complete_data_new.columns.tolist() # convert the names of the headers in the merged data set to a list
                data_to_copy = complete_data_new[header] # data to be moved
                complete_data_new.drop(header, 1, inplace=True) # delete the old copy of the data
                complete_data_new.insert(int(header_loc), header, data_to_copy.values) # insert a copy of the column
                
        # 12. Delete unnecessary columns     
        
        if complete_data_new.shape[1] > column_headings.shape[0]:
            complete_data_new.drop(complete_data_new.columns[list(range(column_headings.shape[0], complete_data_new.shape[1]))], axis=1, inplace=True)
        
        # 13. Save file as complete output TESTING PRINT SIZES OUT
        print('data shape is ' + str(data.shape))
        print('complete_data_new shape is ' + str(complete_data_new.shape))
        
        complete_data_new.to_excel(os.path.join(xlsx_files_root_dir + "/" + output_folder + "/" + str(branch_id) + file_tag), index=False)
        
        # 14. Save branch_id from this loop
        
        branch_id_prev = branch_id
        
        # 14. Delete TMP file
        
        try:
            os.remove(os.path.join(xlsx_files_root_dir + "/" + "tmp" + file_tag))
        except FileNotFoundError:
            pass
        
        # 15. Indicate first run complete 
        
        first_run = False

        
# 16. For final store, apply step 4.
# find unwanted string values cells and delete
for value in values_to_remove:
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = complete_data_new.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        complete_data_new[col] = ''
complete_data_new.columns = column_headings_order
complete_data_new.to_excel(os.path.join(xlsx_files_root_dir + "/" + output_folder + "/" + str(branch_id_prev) + file_tag), index=False)

# 17. Alert user that script has finished
print('Finished')

2709_ElectricityEnvironment_Total.xlsx
data shape is (35040, 7)
complete_data_new shape is (35040, 38)
2709_ElectricityHours_Total.xlsx
data shape is (35040, 6)
complete_data_new shape is (35040, 38)
2709_FoodPrep_Bakery_Electricity.xlsx
data shape is (17538, 4)
complete_data_new shape is (35056, 38)
2709_FoodPrep_Electricity.xlsx
data shape is (17538, 4)
complete_data_new shape is (35056, 38)
2709_FoodPrep_FoodToGo_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35056, 38)
2709_FoodPrep_JSR_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35056, 38)
2709_HVAC_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35056, 38)
2709_HVAC_hvacAHU_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35056, 38)
2709_HVAC_hvacHandV_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35056, 38)
2709_Lighting_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35056, 38)
2709_Lighting_

data shape is (35040, 4)
complete_data_new shape is (35040, 38)
2242_FoodPrep_JSR_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35040, 38)
2242_Gas_Total.xlsx
data shape is (35040, 4)
complete_data_new shape is (35040, 38)
2242_GoodsOnLine_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35040, 38)
2242_HVAC_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35040, 38)
2242_HVAC_hvacAHU_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35040, 38)
2242_HVAC_hvacHandV_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35040, 38)
2242_Lighting_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35040, 38)
2242_Lighting_LightingExternal_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35040, 38)
2242_Lighting_LightingSalesAmbient_Electricity.xlsx
data shape is (35040, 4)
complete_data_new shape is (35040, 38)
2242_Lighting_LightingSalesFeature_Electr

In [24]:
# END OF SCRIPT

,Date,Tenant Electricity consumption (kW),Baseline (kW),Asset status
0,2018-09-10 00:30:00,0,0.612601,Active
1,2018-09-10 01:00:00,2,0.547324,Active
2,2018-09-10 01:30:00,0,0.657793,Active
3,2018-09-10 02:00:00,0,0.602558,Active
4,2018-09-10 02:30:00,0,0.567409,Active
5,2018-09-10 03:00:00,2,0.602558,Active
6,2018-09-10 03:30:00,0,0.597537,Active
7,2018-09-10 04:00:00,0,0.632686,Active
8,2018-09-10 04:30:00,0,0.577452,Active
9,2018-09-10 05:00:00,2,0.637708,Active


In [73]:
complete_data_new

4,Date,Operating hours,Opening hours,Sunshine hours (h),Humidity (%),Temperature (°C),Asset status,Store Electricity consumption (kW),Baseline (kW),Actual (kW),Baseline (kW),Actual (kW),Baseline (kW)
5,2017-09-10 00:30:00,0,0,,,,,166.507,181.626,11.6,18.3388,2,9.5102
6,2017-09-10 01:00:00,0,0,0,94,10,,166.656,186.917,14.8,23.8204,4,12.449
7,2017-09-10 01:30:00,0,0,,,,,165.057,200.293,17.2,55.2458,2,41.9583
8,2017-09-10 02:00:00,0,0,0,100,10,,165.566,208.471,14.8,59.4208,2,46.2083
9,2017-09-10 02:30:00,0,0,,,,,163.747,220.709,16.6,61.3061,6,52.0816
10,2017-09-10 03:00:00,0,0,0,100,10,,161.456,212.651,15.4,57.1551,6,48.6122
11,2017-09-10 03:30:00,0,0,,,,,269.885,215.734,111.8,61.2694,104,52.4082
12,2017-09-10 04:00:00,0,0,0,100,10,,235.505,215.946,81.6,55.1878,72,47.102
13,2017-09-10 04:30:00,0,0,,,,,233.156,222.785,74,53.4694,64,42.449
14,2017-09-10 05:00:00,0,0,0,100,10,,220.366,218.142,59.2,49.0408,50,40.2041


In [31]:
complete_data_new.to_excel(os.path.join(xlsx_files_root_dir, output_folder, str(branch_id) + file_tag), index=False)
        

In [33]:
complete_data_new.to_excel(r'C:\Users\benfl\OneDrive - Flexible Power Systems Ltd\Documents\Load Profiles - PythonJoinTestingDelete\complete_profiles\0885_complete.xlsx')

In [ ]:
## EDIT VERSION

# Parent directory of all excel files 
xlsx_files_root_dir = r"C:\Users\benfl\OneDrive - Flexible Power Systems Ltd\Load Profiles - PythonJoinTestingDelete" 

file_tag = "_complete.xlsx"
output_folder = "complete_profiles"
column_headings_path_file = r"C:\Users\benfl\OneDrive - Flexible Power Systems Ltd\Documents\PCM\Load Profile Headers 2.xlsx" 

# read headings 
column_headings = pd.read_excel(column_headings_path_file,keep_default_na=False)


# NEW - LATEST  (21/01/20)  ensure that dates are properly aligned

column_headings_order = column_headings[column_headings.columns[0]]
data_offset = 6
old_branch_id = str(0)
count = 0
values_to_remove = ['Active', 'Setup']
# Fetches all files in directory and loops 
raw_input_row_location_of_header = 4

In [64]:
column_headings_path_file = r"C:\Users\benfl\OneDrive - Flexible Power Systems Ltd\Documents\FPS Developer\SSL_data_cleaning_v3\Documentation\Profile_Schema.xlsx"
column_headings = pd.read_excel(string_new_headers, keep_default_na=False, index_col=0)
column_headings_order = column_headings_NEW.iloc[0]
new_index = list(range(0,column_headings_order.size))
column_headings_order.index = new_index